<a href="https://colab.research.google.com/github/apchavezr/-Analisis_Grandes_Volumenes_Datos/blob/main/analisis_logs_spark_kafka_actualizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Logs con Kafka, Spark Streaming y KMeans
Este notebook guía el desarrollo de una práctica en la que se simula el procesamiento de logs con herramientas de Big Data.

## Objetivos
- Simular un flujo de logs con archivos `.log` generados artificialmente.
- Implementar un pipeline local con Apache Kafka y Spark Streaming.
- Aplicar clustering KMeans para identificar eventos anómalos.
- Visualizar los resultados mediante Kibana.

## Requisitos
- Apache Kafka
- Apache Spark con PySpark
- Elasticsearch y Kibana
- Python 3.8+
- Librerías: `pyspark`, `pandas`, `matplotlib`


In [1]:
# Simulación de logs artificiales
import random
from datetime import datetime, timedelta

def generar_logs(ruta, n=1000):
    niveles = ['INFO', 'WARN', 'ERROR']
    servicios = ['auth', 'db', 'api', 'cache']
    eventos = ['login', 'logout', 'timeout', 'access_denied', 'read', 'write']

    with open(ruta, 'w') as f:
        for _ in range(n):
            tiempo = datetime.now() - timedelta(seconds=random.randint(0, 3600))
            nivel = random.choice(niveles)
            servicio = random.choice(servicios)
            evento = random.choice(eventos)
            linea = f"{tiempo.isoformat()} [{nivel}] {servicio}: {evento}\n"
            f.write(linea)

generar_logs('logs_simulados.log', 1000)
print("Archivo de logs generado.")

Archivo de logs generado.


**Explicación**: Se inicia una sesión de Spark y se realiza la lectura del archivo `.log`. Luego, se aplican transformaciones para extraer los diferentes campos del registro, como el timestamp, nivel del log, servicio involucrado y evento. Esta estructura es necesaria para poder analizar y modelar los datos.

## Consumo de logs con Spark Streaming desde Kafka
Este paso requiere que Kafka esté configurado y que los logs estén siendo enviados a un tópico llamado `logs-topic`.
En Spark, se puede usar el siguiente código para consumir y transformar los datos:

In [2]:
# Código básico para iniciar Spark Streaming y consumir desde Kafka
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col

spark = SparkSession.builder \
    .appName("Log Analysis") \
    .getOrCreate()

df = spark.read.text("logs_simulados.log")
logs_df = df.withColumn("timestamp", split(col("value"), " ")[0]) \
            .withColumn("nivel", split(col("value"), " ")[1]) \
            .withColumn("servicio", split(col("value"), " ")[2]) \
            .withColumn("evento", split(col("value"), ": ")[1])

logs_df.show(5, truncate=False)

+-----------------------------------------------------+--------------------------+------+--------+-------------+
|value                                                |timestamp                 |nivel |servicio|evento       |
+-----------------------------------------------------+--------------------------+------+--------+-------------+
|2025-04-21T19:43:58.964095 [INFO] auth: access_denied|2025-04-21T19:43:58.964095|[INFO]|auth:   |access_denied|
|2025-04-21T19:54:08.964122 [INFO] auth: access_denied|2025-04-21T19:54:08.964122|[INFO]|auth:   |access_denied|
|2025-04-21T20:05:10.964134 [WARN] db: read           |2025-04-21T20:05:10.964134|[WARN]|db:     |read         |
|2025-04-21T20:32:22.964143 [WARN] auth: access_denied|2025-04-21T20:32:22.964143|[WARN]|auth:   |access_denied|
|2025-04-21T20:07:16.964150 [INFO] auth: logout       |2025-04-21T20:07:16.964150|[INFO]|auth:   |logout       |
+-----------------------------------------------------+--------------------------+------+-------

**Explicación**: Aquí se entrena el modelo KMeans con 3 clústeres (puede ajustarse según el análisis). Luego se aplica el modelo a los datos transformados para predecir a qué grupo pertenece cada registro. El resultado es útil para detectar patrones repetitivos o anómalos dentro del flujo de eventos.

## Aplicación de KMeans para agrupar eventos similares
Se realiza una codificación numérica básica y se aplica el algoritmo de clustering.

In [3]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.clustering import KMeans

indexer = StringIndexer(inputCols=["nivel", "servicio", "evento"],
                        outputCols=["nivel_index", "servicio_index", "evento_index"])
logs_indexed = indexer.fit(logs_df).transform(logs_df)

assembler = VectorAssembler(inputCols=["nivel_index", "servicio_index", "evento_index"], outputCol="features")
dataset = assembler.transform(logs_indexed)

kmeans = KMeans(k=3, seed=1)
modelo = kmeans.fit(dataset)
predicciones = modelo.transform(dataset)
predicciones.select("value", "prediction").show(10, truncate=False)

+-----------------------------------------------------+----------+
|value                                                |prediction|
+-----------------------------------------------------+----------+
|2025-04-21T19:43:58.964095 [INFO] auth: access_denied|2         |
|2025-04-21T19:54:08.964122 [INFO] auth: access_denied|2         |
|2025-04-21T20:05:10.964134 [WARN] db: read           |1         |
|2025-04-21T20:32:22.964143 [WARN] auth: access_denied|2         |
|2025-04-21T20:07:16.964150 [INFO] auth: logout       |0         |
|2025-04-21T19:58:28.964156 [ERROR] db: write         |1         |
|2025-04-21T19:47:32.964162 [INFO] cache: login       |0         |
|2025-04-21T20:34:30.964171 [WARN] auth: access_denied|2         |
|2025-04-21T19:49:23.964182 [WARN] db: read           |1         |
|2025-04-21T20:06:23.964190 [INFO] api: access_denied |2         |
+-----------------------------------------------------+----------+
only showing top 10 rows



## Conclusiones
- La simulación de logs permite crear un entorno controlado para probar algoritmos de detección de anomalías.
- Spark Streaming facilita la lectura y transformación de datos de forma escalable, adecuada para flujos en tiempo real.
- El uso de clustering con KMeans permite identificar agrupaciones de eventos similares, lo que ayuda a clasificar errores o comportamientos inesperados.
- Integrar este pipeline con herramientas de visualización como Kibana permite a los equipos de operaciones y seguridad monitorear los sistemas en tiempo real de forma visual e interactiva.